<a href="https://colab.research.google.com/github/Kamani-Shivani/NLP/blob/main/Assignment_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use a small dataset of English to French sentence pairs. You can replace this with any other language pair dataset. data = [ ("hello", "bonjour"), ("how are you", "comment ça va"), ("I am fine", "je vais bien"), ("what is your name", "comment tu t'appelles"), ("my name is", "je m'appelle"), ("thank you", "merci"), ("goodbye", "au revoir") ] [CO5]

(a) Take input (English) and target (French)

In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

data = [
    ("hello", "bonjour"),
    ("how are you", "comment ça va"),
    ("I am fine", "je vais bien"),
    ("what is your name", "comment tu t'appelles"),
    ("my name is", "je m'appelle"),
    ("thank you", "merci"),
    ("goodbye", "au revoir")
]

input_texts = [pair[0] for pair in data]
target_texts = ['\t' + pair[1] + '\n' for pair in data]

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    print(f"Example {i+1}:")
    print(f"Input: {input_text}")
    print(f"Target: {target_text}")
    print()

Example 1:
Input: hello
Target: 	bonjour


Example 2:
Input: how are you
Target: 	comment ça va


Example 3:
Input: I am fine
Target: 	je vais bien


Example 4:
Input: what is your name
Target: 	comment tu t'appelles


Example 5:
Input: my name is
Target: 	je m'appelle


Example 6:
Input: thank you
Target: 	merci


Example 7:
Input: goodbye
Target: 	au revoir




(b) Building the Seq2Seq Model

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

data = [
    ("hello", "bonjour"),
    ("how are you", "comment ça va"),
    ("I am fine", "je vais bien"),
    ("what is your name", "comment tu t'appelles"),
    ("my name is", "je m'appelle"),
    ("thank you", "merci"),
    ("goodbye", "au revoir")
]

input_texts = [pair[0] for pair in data]
target_texts = ['\t' + pair[1] + '\n' for pair in data]

input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(input_texts)
input_seq = input_tokenizer.texts_to_sequences(input_texts)

target_tokenizer = Tokenizer()
target_tokenizer.fit_on_texts(target_texts)
target_seq = target_tokenizer.texts_to_sequences(target_texts)

input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

max_input_length = max(len(seq) for seq in input_seq)
max_target_length = max(len(seq) for seq in target_seq)

input_seq = pad_sequences(input_seq, maxlen=max_input_length, padding='post')
target_seq = pad_sequences(target_seq, maxlen=max_target_length, padding='post')

decoder_input_seq = target_seq[:, :-1]
decoder_output_seq = target_seq[:, 1:]

decoder_output_seq_onehot = np.zeros((len(decoder_output_seq), max_target_length - 1, target_vocab_size), dtype="float32")
for i, seq in enumerate(decoder_output_seq):
    for t, word in enumerate(seq):
        decoder_output_seq_onehot[i, t, word - 1] = 1.0

encoder_inputs = Input(shape=(max_input_length,))
encoder_embedding = Embedding(input_vocab_size, 256)(encoder_inputs)
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(max_target_length - 1,))
decoder_embedding = Embedding(target_vocab_size, 256)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_lstm_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(target_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

model.fit([input_seq, decoder_input_seq], decoder_output_seq_onehot, epochs=10, batch_size=2, validation_split=0.2)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 4)              │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_1             │ (None, 2)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 4, 256)         │          3,840 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_1 (Embedding)   │ (None, 2, 256)         │          3,584 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ [(None, 256), (None,   │        525,312 │ embedding[0][0]        │
│                           │ 256), (None, 256)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ [(None, 2, 256),       │        525,312 │ embedding_1[0][0],     │
│                           │ (None, 256), (None,    │                │ lstm[0][1], lstm[0][2] │
│                           │ 256)]                  │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 2, 14)          │          3,598 │ lstm_1[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,061,646 (4.05 MB)

 Trainable params: 1,061,646 (4.05 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 397ms/step - accuracy: 0.0500 - loss: 2.6431 - val_accuracy: 0.7500 - val_loss: 2.5404
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - accuracy: 0.3750 - loss: 2.5552 - val_accuracy: 0.7500 - val_loss: 2.4465
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - accuracy: 0.3438 - loss: 2.4795 - val_accuracy: 0.7500 - val_loss: 2.3051
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.6250 - loss: 2.2963 - val_accuracy: 0.7500 - val_loss: 2.0202
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.5000 - loss: 2.0595 - val_accuracy: 0.7500 - val_loss: 1.6046
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.5312 - loss: 1.9828 - val_accuracy: 0.7500 - val_loss: 1.4147
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.1813 - loss: 1.9087 - val_accuracy: 0.7500 - val_loss: 1.7841
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.3250 - loss: 1.8340 - val_accuracy: 0.7500 - val_loss: 2.1861

In [3]:
model.fit([input_seq, decoder_input_seq], decoder_output_seq_onehot, epochs=10, batch_size=2, validation_split=0.2)

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.9500 - loss: 1.2040 - val_accuracy: 0.7500 - val_loss: 2.3069
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.8562 - loss: 0.8871 - val_accuracy: 0.7500 - val_loss: 2.4939
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.9500 - loss: 0.5696 - val_accuracy: 0.7500 - val_loss: 2.7322
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.9500 - loss: 0.3780 - val_accuracy: 0.7500 - val_loss: 3.0431
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - accuracy: 1.0000 - loss: 0.3612 - val_accuracy: 0.7500 - val_loss: 3.3459
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 1.0000 - loss: 0.2676 - val_accuracy: 0.7500 - val_loss: 3.6049
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 1.0000 - loss: 0.2157 - val_accuracy: 0.7500 - val_loss: 3.8107
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 1.0000 - loss: 0.1454 - val_accuracy: 0.7500 - val_loss: 4.0158


#### (d) After training, set up separate models for the encoder and decoder to perform inference (translation) on new sentences.

In [4]:

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]


decoder_lstm_output, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

decoder_output = decoder_dense(decoder_lstm_output)

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_output] + decoder_states)


(c) Prepare the target sequences for training by shifting them by one position, which the model will use to predict the next word in the sequence

In [8]:
def translate(input_text):

    input_seq = input_tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=max_input_length, padding='post')

    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_tokenizer.word_index['\t']

    translated_sentence = ''

    for _ in range(max_target_length - 1):

        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])

        sampled_word = target_tokenizer.index_word.get(sampled_token_index, '')

        print(f"Sampled token index: {sampled_token_index}, word: {sampled_word}")

        if sampled_word == '\n':
            break

        translated_sentence += ' ' + sampled_word

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return translated_sentence.strip()

input_sentence = "hello"
translated_sentence = translate(input_sentence)

print(f"Input: {input_sentence}")
print(f"Translation: {translated_sentence}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Sampled token index: 15, word: 

Input: hello
Translation: 


(e) Use the trained model to translate new English sentences into French

In [7]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

data = [
    ("hello", "bonjour"),
    ("how are you", "comment ça va"),
    ("I am fine", "je vais bien"),
    ("what is your name", "comment tu t'appelles"),
    ("my name is", "je m'appelle"),
    ("thank you", "merci"),
    ("goodbye", "au revoir")
]

input_texts = [pair[0] for pair in data]
target_texts = ['\t' + pair[1] + '\n' for pair in data]

input_tokenizer = Tokenizer()
input_tokenizer.fit_on_texts(input_texts)
input_seq = input_tokenizer.texts_to_sequences(input_texts)

target_tokenizer = Tokenizer()
target_tokenizer.fit_on_texts(target_texts)
target_seq = target_tokenizer.texts_to_sequences(target_texts)

target_tokenizer.word_index['\t'] = len(target_tokenizer.word_index) + 1
target_tokenizer.word_index['\n'] = len(target_tokenizer.word_index) + 1

target_tokenizer.index_word[target_tokenizer.word_index['\t']] = '\t'
target_tokenizer.index_word[target_tokenizer.word_index['\n']] = '\n'

input_vocab_size = len(input_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

max_input_length = max(len(seq) for seq in input_seq)
max_target_length = max(len(seq) for seq in target_seq)

input_seq = pad_sequences(input_seq, maxlen=max_input_length, padding='post')
target_seq = pad_sequences(target_seq, maxlen=max_target_length, padding='post')

decoder_input_seq = target_seq[:, :-1]
decoder_output_seq = target_seq[:, 1:]

decoder_output_seq_onehot = np.zeros((len(decoder_output_seq), max_target_length - 1, target_vocab_size), dtype="float32")
for i, seq in enumerate(decoder_output_seq):
    for t, word in enumerate(seq):
        decoder_output_seq_onehot[i, t, word - 1] = 1.0

encoder_inputs = Input(shape=(max_input_length,))
encoder_embedding = Embedding(input_vocab_size, 256)(encoder_inputs)
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(max_target_length-1,))
decoder_embedding = Embedding(target_vocab_size, 256)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_lstm_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(target_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

model.fit([input_seq, decoder_input_seq], decoder_output_seq_onehot, epochs=10, batch_size=2, validation_split=0.2, callbacks=[early_stopping])

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm_output, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

decoder_output = decoder_dense(decoder_lstm_output)

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_output] + decoder_states)

def translate(input_text):

    input_seq = input_tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=max_input_length, padding='post')

    states_value = encoder_model.predict(input_seq)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_tokenizer.word_index['\t']

    translated_sentence = ''

    for _ in range(max_target_length - 1):

        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = target_tokenizer.index_word[sampled_token_index]

        if sampled_word == '\n':
            break

        translated_sentence += ' ' + sampled_word

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return translated_sentence.strip()

for input_sentence, target_sentence in zip(input_texts, target_texts):
    translated_sentence = translate(input_sentence)
    print(f"Input (English): {input_sentence}")
    print(f"Actual Translation (French): {target_sentence.strip()}")
    print(f"Translated Sentence (French): {translated_sentence}")
    print('-' * 40)

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 4)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_5             │ (None, 2)              │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, 4, 256)         │          3,840 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, 2, 256)         │          4,096 │ input_layer_5[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_2 (LSTM)             │ [(None, 256), (None,   │        525,312 │ embedding_2[0][0]      │
│                           │ 256), (None, 256)]     │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_3 (LSTM)             │ [(None, 2, 256),       │        525,312 │ embedding_3[0][0],     │
│                           │ (None, 256), (None,    │                │ lstm_2[0][1],          │
│                           │ 256)]                  │                │ lstm_2[0][2]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 2, 16)          │          4,112 │ lstm_3[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 1,062,672 (4.05 MB)

 Trainable params: 1,062,672 (4.05 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 263ms/step - accuracy: 0.1437 - loss: 2.7668 - val_accuracy: 0.7500 - val_loss: 2.6890
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.4688 - loss: 2.6820 - val_accuracy: 0.7500 - val_loss: 2.6009
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - accuracy: 0.3875 - loss: 2.5785 - val_accuracy: 0.7500 - val_loss: 2.4369
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.2625 - loss: 2.4850 - val_accuracy: 0.7500 - val_loss: 2.2151
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.5125 - loss: 2.1515 - val_accuracy: 0.7500 - val_loss: 1.8178
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.3688 - loss: 1.8307 - val_accuracy: 0.7500 - val_loss: 1.5480
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.2125 - loss: 1.8512 - val_accuracy: 0.7500 - val_loss: 1.7788
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.3688 - loss: 1.3679 - val_accuracy: 0.7500 - val_loss: 2.1686